# Analyze the structural information of functional variants

In [12]:
import requests
import glob
import gzip
import re
import os
import numpy as np
import py3Dmol
import polars as pl
from Bio.PDB import PDBParser
from sklearn.cluster import AgglomerativeClustering

In [2]:
def get_uniprot_swissprot_id(protein_name: str) -> str:
    """
    Query UniProt’s REST search API to find the reviewed (Swiss‐Prot) accession
    for a given human gene/protein name. Returns None if not found.
    """
    url = "https://rest.uniprot.org/uniprotkb/search"
    # Build a query that:
    #  - matches the gene name exactly (using “gene:”)
    #  - restricts to human (organism_id:9606)
    #  - restricts to reviewed (Swiss‐Prot) entries
    query = f"gene:{protein_name} AND organism_id:9606 AND reviewed:true"
    params = {
        "query": query,
        "fields": "accession",
        "format": "json",
        "size": 1,      # only need the top hit
    }
    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        results = data.get("results", [])
        if not results:
            return None
        return results[0]["primaryAccession"]
    except Exception:
        # If the request fails (e.g. no internet), return None
        return None

UniProt Swiss‐Prot ID for RHO: P08100


## 1. Loading alleles info

In [13]:
PLATEMAP_DIR = "../../2_individual_assay_analyses/imaging/1_inputs/1_snakemake_pipeline/inputs/metadata/platemaps/{batch_id}/platemap"
BIO_REP_BATCHES = ["2025_01_27_Batch_13", "2025_01_28_Batch_14"]

allele_meta_dict = {}
for batch_id in BIO_REP_BATCHES:
    allele_meta_df = pl.DataFrame()
    for platemap in os.listdir(PLATEMAP_DIR.format(batch_id=batch_id)):
        platemap_df = pl.read_csv(os.path.join(PLATEMAP_DIR.format(batch_id=batch_id), platemap), separator="\t", infer_schema_length=100000)
        allele_meta_df = pl.concat([allele_meta_df, 
                                    platemap_df.filter((pl.col("node_type").is_not_null()))], # (~pl.col("node_type").is_in(["TC","NC","PC"]))&
                                    how="diagonal_relaxed").sort("plate_map_name")
        allele_meta_df = allele_meta_df.with_columns(pl.col("plate_map_name").str.split('_').list.get(0).alias("plate_map"))
    allele_meta_dict[batch_id] = allele_meta_df

In [4]:
gene = "RHO"
gene_uniprot_id = get_uniprot_swissprot_id(gene)
print(f"UniProt Swiss‐Prot ID for {gene}:", {gene_uniprot_id} or "Not found")

af2_struc_pdb_dir = "/data/shenrunx/igvf/varchamp/2025_laval_submitted/compare_ai_scores/1_inputs/new_raw_data/alphafold/UP000005640_9606_HUMAN_v4"

pdb_file = [pdb for pdb in glob.glob(f"{af2_struc_pdb_dir}/*.pdb.gz") if gene_uniprot_id in pdb][0]
pdb_file

UniProt Swiss‐Prot ID for RHO: {'P08100'}


'/data/shenrunx/igvf/varchamp/2025_laval_submitted/compare_ai_scores/1_inputs/new_raw_data/alphafold/UP000005640_9606_HUMAN_v4/AF-P08100-F1-model_v4.pdb.gz'

In [21]:
# 1. Parse the structure
# 1) Open the .pdb.gz file with gzip.open in text/binary mode:
with gzip.open(pdb_file, "rt") as handle:
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("rho", handle)

    # 3. Rewind and read the entire PDB text
    handle.seek(0)
    pdb_text = handle.read()

# 2. Build a map: residue_number → (x, y, z) of its Cα atom
ca_coords = {}
for model in structure:
    for chain in model:
        if chain.get_id() == "A":
            for residue in chain:
                resnum = residue.get_id()[1]
                # Only consider standard amino acids with a CA atom
                if 'CA' in residue:
                    atom_ca = residue['CA']
                    ca_coords[resnum] = atom_ca.get_coord()

# 2) List of variants in “RHO_AminoAcidResidueNumberNewAminoAcid” format.
#    We assume that the PDB’s residue numbering matches exactly the numbers in these strings.
variants_raw = allele_meta_df.filter(pl.col("gene_allele").str.contains(f"{gene}_"))["gene_allele"].unique()
# 3) The chain ID in the PDB that contains Rhodopsin.
#    If your PDB uses a different chain (e.g. “B”), adjust accordingly.
chain_id = "A"
# 4) If your PDB has insertion codes (e.g. 106A, 106B), you must handle that.
#    In this example, we assume no insertion codes (icode = " ").
default_icode = " "
# Confirm chain exists
model = structure[0]  # first (and usually only) model
if chain_id not in model:
    raise ValueError(f"Chain '{chain_id}' not found in {pdb_file}.")
# ---- STEP 2: Convert each raw‐string into (chain, resi, icode, label) ----
variant_list = []
pattern = re.compile(rf"{gene}_([A-Za-z]{{3}})(\d+)([A-Za-z]{{3}})")
for raw in variants_raw:
    m = pattern.match(raw)
    if not m:
        print(f"Warning: '{raw}' does not match expected pattern 'RHO_Aaa###Bbb'. Skipping.")
        continue
    wt_3letter, resi_str, mut_3letter = m.groups()
    resi = int(resi_str)
    # Convert three‐letter codes to one‐letter for a concise label if you like.
    # Here, we'll keep the original string as the label.
    label = raw
    variant_list.append({
        "chain": chain_id,
        "resi": resi,
        "icode": default_icode,
        "label": label
    })

print(variant_list)
if not variant_list:
    raise RuntimeError("No valid variants found. Please check your variant strings.")

RHO_Arg135Leu
RHO
RHO_Pro267Thr
RHO_Cys110Tyr
RHO_Asp190Gly
RHO_Arg135Trp
[{'chain': 'A', 'resi': 135, 'icode': ' ', 'label': 'RHO_Arg135Leu'}, {'chain': 'A', 'resi': 267, 'icode': ' ', 'label': 'RHO_Pro267Thr'}, {'chain': 'A', 'resi': 110, 'icode': ' ', 'label': 'RHO_Cys110Tyr'}, {'chain': 'A', 'resi': 190, 'icode': ' ', 'label': 'RHO_Asp190Gly'}, {'chain': 'A', 'resi': 135, 'icode': ' ', 'label': 'RHO_Arg135Trp'}]


In [37]:
# Define 3D offsets for each residue
offsets_3d = {
    135: {'x': +1.0, 'y': +0.5, 'z': +0.5},
    110: {'x': -1.0, 'y': +0.5, 'z': +0.5},
    190: {'x':  0.0, 'y': -1.0, 'z': +0.5}
}

view = py3Dmol.view(width=800, height=600)
view.addModel(pdb_text, 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})

residues_to_label = [var['resi'] for var in variant_list]
# Get atom coordinates first to calculate proper offsets
for i, resi in enumerate(residues_to_label):
    view.setStyle({'chain': 'A', 'resi': resi}, {'sphere': {'color': 'red', 'radius': .5}})
    # Add a dummy atom at offset position for labeling
    # This requires knowing the actual coordinates
    view.addResLabels(
        {'chain': 'A', 'resi': residues_to_label},
        {'fontColor': 'red', 'fontSize': 15, 'showBg': False}
    )
    
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [20]:
# ---- STEP 3: Extract Cα coordinates for each variant residue ----
coords = []
labels = []
missing = []

for var in variant_list:
    chn = var["chain"]
    resi = var["resi"]
    icode = var["icode"]
    label = var["label"]
    try:
        residue = model[chn][(" ", resi, icode)]
        ca_atom = residue["CA"]
        ca_coord = ca_atom.get_coord()  # NumPy array [x, y, z]
        coords.append(ca_coord)
        labels.append(label)
    except KeyError:
        missing.append(label)

if missing:
    print("Warning: The following variants were not found in chain "
          f"{chain_id}, PDB numbering. Skipping:")
    for lbl in missing:
        print("  ", lbl)
if not coords:
    raise RuntimeError("None of the requested variant residues could be found in the structure.")

coords_array = np.vstack(coords)  # shape = (N_found, 3)
print(coords_array)

# ---- STEP 4: Cluster the Cα coordinates in 3D ----
# Example: Agglomerative clustering into 2 clusters. Adjust n_clusters as desired.
n_clusters = 2
clust = AgglomerativeClustering(
    n_clusters=n_clusters,
    metric="euclidean",
    linkage="average"
)

cluster_labels = clust.fit_predict(coords_array)

# ---- STEP 5: Report cluster assignments ----

clusters = {}
for lbl, cidx in zip(labels, cluster_labels):
    clusters.setdefault(cidx, []).append(lbl)
print("\nCluster assignments for RHO variants (chain A):")
for cidx, var_names in clusters.items():
    print(f" Cluster {cidx + 1}: {var_names}")

RHO_Arg135Trp
RHO_Pro267Thr
RHO_Arg135Leu
RHO_Cys110Tyr
RHO
RHO_Asp190Gly
[{'chain': 'A', 'resi': 135, 'icode': ' ', 'label': 'RHO_Arg135Trp'}, {'chain': 'A', 'resi': 267, 'icode': ' ', 'label': 'RHO_Pro267Thr'}, {'chain': 'A', 'resi': 135, 'icode': ' ', 'label': 'RHO_Arg135Leu'}, {'chain': 'A', 'resi': 110, 'icode': ' ', 'label': 'RHO_Cys110Tyr'}, {'chain': 'A', 'resi': 190, 'icode': ' ', 'label': 'RHO_Asp190Gly'}]
[[-17.046 -10.497  10.764]
 [ -5.22    3.693  -8.843]
 [-17.046 -10.497  10.764]
 [ 15.914   1.482  -2.708]
 [  9.402  -3.538 -11.087]]

Cluster assignments for RHO variants (chain A):
 Cluster 2: ['RHO_Arg135Trp', 'RHO_Arg135Leu']
 Cluster 1: ['RHO_Pro267Thr', 'RHO_Cys110Tyr', 'RHO_Asp190Gly']


In [ ]:
## binding with transporters